In [50]:
import time
from collections import Counter
from PIL import Image, ImageDraw
import os
import pyautogui

In [51]:
l = 20
h = 14

base_x = 235
base_y = 5

step_x = 130
step_y = 65

box_size = 30

dict_swap_dofus_to_pos = {}
dict_swap_pos_to_dofus  = {}

In [52]:
x_start = 1
y_start = 13
for j in range(h):
    x_start = j
    y_start = h - j - 1
    for i in range(l):
        dict_swap_dofus_to_pos[(i, y_start, False)] = (x_start + i, y_start + i)
        dict_swap_dofus_to_pos[(i, y_start, True)] = (x_start + i, y_start + i + 1)
        
        dict_swap_pos_to_dofus[(x_start + i, y_start + i)] = (i, y_start, False)
        dict_swap_pos_to_dofus[(x_start + i, y_start + i + 1)] = (i, y_start, True)


In [53]:
print(dict_swap_dofus_to_pos)

{(0, 13, False): (0, 13), (0, 13, True): (0, 14), (1, 13, False): (1, 14), (1, 13, True): (1, 15), (2, 13, False): (2, 15), (2, 13, True): (2, 16), (3, 13, False): (3, 16), (3, 13, True): (3, 17), (4, 13, False): (4, 17), (4, 13, True): (4, 18), (5, 13, False): (5, 18), (5, 13, True): (5, 19), (6, 13, False): (6, 19), (6, 13, True): (6, 20), (7, 13, False): (7, 20), (7, 13, True): (7, 21), (8, 13, False): (8, 21), (8, 13, True): (8, 22), (9, 13, False): (9, 22), (9, 13, True): (9, 23), (10, 13, False): (10, 23), (10, 13, True): (10, 24), (11, 13, False): (11, 24), (11, 13, True): (11, 25), (12, 13, False): (12, 25), (12, 13, True): (12, 26), (13, 13, False): (13, 26), (13, 13, True): (13, 27), (14, 13, False): (14, 27), (14, 13, True): (14, 28), (15, 13, False): (15, 28), (15, 13, True): (15, 29), (16, 13, False): (16, 29), (16, 13, True): (16, 30), (17, 13, False): (17, 30), (17, 13, True): (17, 31), (18, 13, False): (18, 31), (18, 13, True): (18, 32), (19, 13, False): (19, 32), (19, 

In [54]:
def crop_image(image, rect, path):
    region = image.crop(rect)
    region.save(path)

In [55]:
def full_crop_image(image_name):
    img = Image.open(f"data/fight/{image_name}.png")
    os.mkdir(f"data/cropped/{image_name}")
    
    draw = ImageDraw.Draw(img)
    
    for i in range(h):
        for j in range(l):
            x = base_x + step_x * i
            y = base_y + step_y * j
            
            crop_image(img, (x, y, x + box_size, y + box_size), f"data/cropped/{image_name}/{image_name}-{i}-{j}-false.png")
            draw.rectangle((x, y, x + box_size, y + box_size), outline="red", width=2)
            
            x += step_x // 2
            y += step_y // 2
            crop_image(img, (x, y, x + box_size, y + box_size), f"data/cropped/{image_name}/{image_name}-{i}-{j}-true.png")
            draw.rectangle((x, y, x + box_size, y + box_size), outline="red", width=2)

    
    img.save(f"data/cropped_visu/{image_name}.png")
    
    img.close()

In [56]:
#folder_dir = "data/fight/"
#for images in os.listdir(folder_dir):
#    if images.endswith(".png"):
#        full_crop_image(images.split('.')[0])

In [57]:
def get_colors(path=None, img=None):
    
    if img is None:
        img = Image.open(path)
        
    pixels = list(img.getdata())

    colors_counter = Counter(pixels)
    colors_counter = sorted(colors_counter.items(), key=lambda x: x[1], reverse=True)

    color1, _ = colors_counter[0]    
    if len(colors_counter) > 1:
        color2, _ = colors_counter[1]
    else :
        color2 = (0, 0, 0)
        
    return color1, color2

In [58]:
list_color = {}

for image in os.listdir("./data/cropped/139"):
    if image.endswith(".png"):
        color = get_colors("./data/cropped/139/" + image)
        if color[0] not in list_color.keys():
            list_color[color[0]] = image
            print(f"{image} -> {color}")


139-8-6-false.png -> ((142, 134, 94, 255), (88, 83, 58, 255))
139-6-2-false.png -> ((0, 0, 0, 255), (0, 0, 0))
139-12-11-false.png -> ((88, 83, 58, 255), (89, 84, 59, 255))
139-7-9-true.png -> ((150, 142, 103, 255), (88, 83, 58, 255))
139-5-11-true.png -> ((218, 57, 45, 255), (230, 127, 46, 255))
139-2-12-true.png -> ((46, 54, 61, 255), (77, 77, 93, 255))
139-8-9-false.png -> ((251, 241, 191, 255), (46, 54, 61, 255))
139-6-17-false.png -> ((196, 19, 0, 255), (101, 11, 1, 255))


In [59]:
EMPTY_CELL = 0
WALL = 1
USABLE_CELL = 2
PLAYER = 3
MONSTER = 4
INVO = 5
BOSS = 6
UNDEFINED = -1

list_cell_color = {
    (0, 0, 0, 255): EMPTY_CELL,
    (142, 134, 94): USABLE_CELL,
    (150, 142, 103): USABLE_CELL,
    (88, 83, 58): WALL,
    (218, 57, 45): INVO,
    (46, 54, 61): MONSTER,
    (251, 241, 191): BOSS,
    (196, 19, 0): PLAYER,
}


In [60]:
def get_image_type(path=None, img=None):
    
    if img is None:
        most = get_colors(path=path)
    else:
        most = get_colors(img=img)
    
    type = UNDEFINED
    
    if most[0] in list_color.keys():
        type = list_color[most[0]]
    if type == USABLE_CELL and most[1] == (46, 54, 61):
        type = MONSTER
        
    return type

In [61]:
def get_screen_info():
    
    dist_case = {}
    
    screen = pyautogui.screenshot()
    
    for pos in dict_swap_pos_to_dofus.keys():
        
        dofus_pos = dict_swap_pos_to_dofus[pos]
        x = base_x + step_x * dofus_pos[0]
        y = base_y + step_y * dofus_pos[1]
        
        if dofus_pos[2]:
            x += step_x // 2
            y += step_y // 2
        
        cropped = screen.crop((x, y, x + box_size, y + box_size))
        
        dist_case[pos] = get_image_type(img=cropped)
        
    return dist_case

In [62]:
while True:
    time.sleep(1)
    get_screen_info()

KeyboardInterrupt: 